<a href="https://colab.research.google.com/github/D-Yadav/NLP/blob/main/Fake_News_Classifier_using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fake News Classifier Using LSTM
Dataset: https://www.kaggle.com/c/fake-news/data#

**Here's Key-takeways what is done in this model : **
1. Dataset
2. Independent and dependent features
3. Cleaning the data (Stemming, & Stopwords)
4. Fix the sentence length to fix the input
5. One Hot Representation, Embedding Layer
6. LSTM neural network



In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/train.csv', error_bad_lines=False, engine='python')

<ipython-input-6-eaff4f369a85>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('/content/train.csv', error_bad_lines=False, engine='python')
Skipping line 5544: unexpected end of data


In [ ]:
df.head() #Retreiving top 5 records

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
df.isnull().sum() #Checking for null values

id          0
title     152
author    541
text       11
label       0
dtype: int64

In [ ]:
# Dropping the null-values
df = df.dropna()

In [ ]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
## Get Independent Features
X = df.drop('label',axis=1)

In [16]:
## Get the dependent features
y =df['label']

In [17]:
X.shape

(4849, 4)

In [18]:
y.shape

(4849,)

In [19]:
import tensorflow as tf
tf.__version__

'2.12.0'

In [21]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [22]:
### Vocabulary Size
voc_size=5000

## One-Hot Representation

In [23]:
messages=X.copy()

In [24]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [27]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
5537,5537,4 Ways to Cut the Duration of Your Cold,Heather Callaghan,"By Pamela Bofferding At our house, back to sch..."
5538,5538,Trump Spokesperson Katrina Pierson Caught In B...,Andrew Bradford,Trump Spokesperson Katrina Pierson Caught In B...
5539,5539,Anonymous World War 3 Has Begun!,Pakalert,source Add To The Conversation Using Facebook ...
5540,5540,CNN Fact Checks Sean Spicer Joke About Salad D...,Ezra Dulis,CNN has published a stinging fact check of the...


In [29]:
messages.reset_index(inplace=True)

In [30]:
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
4844,5537,5537,4 Ways to Cut the Duration of Your Cold,Heather Callaghan,"By Pamela Bofferding At our house, back to sch..."
4845,5538,5538,Trump Spokesperson Katrina Pierson Caught In B...,Andrew Bradford,Trump Spokesperson Katrina Pierson Caught In B...
4846,5539,5539,Anonymous World War 3 Has Begun!,Pakalert,source Add To The Conversation Using Facebook ...
4847,5540,5540,CNN Fact Checks Sean Spicer Joke About Salad D...,Ezra Dulis,CNN has published a stinging fact check of the...


In [31]:
import nltk
import re #regular expression
from nltk.corpus import stopwords

In [33]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [35]:
## Data Preprocessing
from nltk.stem.porter import PorterStemmer ##Stemming Purpose
ps = PorterStemmer() # Initialize Class to Object
corpus = [] # Defined a list which is called as corpus
for i in range (0, len(messages)): # Traversing through each and every data-points inside the messages then selecting column 'title', applying stemming on every words and then 'stopwords'
  #print(i)
  review = re.sub('[^a-zA-Z]',' ', messages['title'][i]) #apart from a-z and A-Z, removing all necessary special characters with ' '
  review = review.lower() # Lowercasing
  review = review.split() # Splitting to gets word by word

  review = [ps.stem(word) for word in review if not word in stopwords.words('english')] # Traversing using for loop in list comprehension and if word not present in english stopwords then apply stemming, if word is present just ignore it
  review = ' '.join(review) # Join all the review
  corpus.append(review) # Combine review to create corpus

In [36]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [37]:
### we can also apply lemmetization but we're ignoring that for now since it take time for execution

In [38]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [39]:
onehot_repr = [one_hot(words, voc_size)for words in corpus] # Traversing through every word in corpus in that particular voc_size and applying one_hot encoding
onehot_repr

[[522, 4983, 208, 2002, 2091, 1391, 3745, 4024, 495, 4922],
 [2406, 2789, 548, 3902, 1421, 4894, 494],
 [4613, 1495, 212, 2964],
 [4690, 1414, 1091, 517, 2933, 4193],
 [290, 1421, 2043, 842, 4934, 1080, 1421, 384, 4333, 4802],
 [3433,
  1616,
  3340,
  4951,
  1399,
  4058,
  1685,
  1296,
  4137,
  272,
  3762,
  1353,
  4754,
  3814,
  494],
 [2297, 4346, 2034, 1900, 199, 381, 3636, 1499, 4372, 2991, 907],
 [3056, 2852, 4034, 1828, 3011, 1112, 4058, 2596, 4372, 2991, 907],
 [2930, 515, 822, 80, 3791, 4330, 1045, 3090, 4058, 161],
 [436, 4191, 1374, 2440, 3566, 2599, 2614, 4969],
 [491, 110, 4035, 1474, 4938, 4797, 947, 3317, 1402, 1613, 1492],
 [517, 82, 2091, 4330, 4058, 3011],
 [1483, 3902, 1873, 2355, 1435, 1396, 2088, 1647, 4116],
 [63, 1174, 742, 4404, 3183, 1283, 2657, 4372, 2991, 907],
 [1393, 1397, 53, 4519, 2596, 4372, 2991, 907],
 [680, 4706, 1031, 1613, 3945, 183, 2722, 347, 1281, 2085],
 [4307, 1652, 2789],
 [422, 2173, 1839, 1234, 4058, 245, 171, 494],
 [3039, 3219, 548,

In [40]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [41]:
onehot_repr[1]

[2406, 2789, 548, 3902, 1421, 4894, 494]

## Embedding Representation

In [42]:
sent_length = 20
embedded_docs = pad_sequences(onehot_repr, padding = 'pre', maxlen = sent_length)
print(embedded_docs)

[[   0    0    0 ... 4024  495 4922]
 [   0    0    0 ... 1421 4894  494]
 [   0    0    0 ... 1495  212 2964]
 ...
 [   0    0    0 ...  645 4182 2911]
 [   0    0    0 ... 1327  937  494]
 [   0    0    0 ... 3839 2251 3838]]


In [43]:
embedded_docs[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0, 2406, 2789,  548, 3902, 1421, 4894,  494], dtype=int32)

In [44]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  522,
       4983,  208, 2002, 2091, 1391, 3745, 4024,  495, 4922], dtype=int32)

In [45]:
## Creating model
embedding_vector_features=40 ##feature representation
model=Sequential() #SequentialModel
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length)) # Add embedding layer with parameters voc_size,  embedding features in 40 dim as indicated above, and input len as sent len
model.add(LSTM(100)) #Taken LSTM with 100 neurons
model.add(Dense(1,activation='sigmoid')) #O/p is binary so using activation function as Sigmoid
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy']) #Since o/p is binary hence using loss func as 'binarycrossentropy'
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [46]:
len(embedded_docs),y.shape

(4849, (4849,))

In [48]:
import numpy as np
X_final = np.array(embedded_docs) # Since embedded_docs which is in list is converted to array
y_final = np.array(y)

In [49]:
X_final.shape, y_final.shape

((4849, 20), (4849,))

In [50]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_final,y_final,test_size=0.33,random_state=42)

**Model Training**

In [60]:
## Finally Training
model.fit(X_train, y_train, validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
51/51 [==============================] - 3s 34ms/step - loss: 0.6423 - accuracy: 0.6345 - val_loss: 0.5479 - val_accuracy: 0.7670
Epoch 2/10
51/51 [==============================] - 2s 47ms/step - loss: 0.3873 - accuracy: 0.8334 - val_loss: 0.3581 - val_accuracy: 0.8301
Epoch 3/10
51/51 [==============================] - 1s 27ms/step - loss: 0.1993 - accuracy: 0.9249 - val_loss: 0.3026 - val_accuracy: 0.8695
Epoch 4/10
51/51 [==============================] - 1s 24ms/step - loss: 0.1186 - accuracy: 0.9627 - val_loss: 0.2897 - val_accuracy: 0.8807
Epoch 5/10
51/51 [==============================] - 1s 23ms/step - loss: 0.0774 - accuracy: 0.9766 - val_loss: 0.3421 - val_accuracy: 0.8726
Epoch 6/10
51/51 [==============================] - 1s 24ms/step - loss: 0.0609 - accuracy: 0.9846 - val_loss: 0.3986 - val_accuracy: 0.8632
Epoch 7/10
51/51 [==============================] - 1s 23ms/step - loss: 0.0351 - accuracy: 0.9889 - val_loss: 0.4774 - val_accuracy: 0.8601
Epoch 8/10
51

#### **Adding DropOut**
Dropout will reduce 'Overfitting'

In [59]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features = 10
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0,3)) # This meaning 30% of LSTM after training will be disabled
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

### Performance Metrics and Accuracy

In [61]:
y_pred=model.predict(X_test)

51/51 [==============================] - 1s 5ms/step


In [62]:
y_pred = np.where(y_pred > 0.5, 1,0) ## we can also use AUC ROC Curve

In [54]:
from sklearn.metrics import confusion_matrix

In [63]:
confusion_matrix(y_test, y_pred)

array([[789, 103],
       [ 86, 623]])

In [64]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.8819487820112429

In [65]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.90      0.88      0.89       892
           1       0.86      0.88      0.87       709

    accuracy                           0.88      1601
   macro avg       0.88      0.88      0.88      1601
weighted avg       0.88      0.88      0.88      1601

